<a href="https://colab.research.google.com/github/amalsalilan/Infosys-Springboard-Virtual-Internship-6.0-Open-Deep-Researcher-batch-2/blob/Prateek_Ray/clarifyBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip uninstall -y google-generativeai google-ai-generativelanguage
!pip install google-generativeai==0.8.5 google-ai-generativelanguage==0.7.0
!pip install --upgrade langchain-google-genai langchain

Found existing installation: google-generativeai 0.8.5
Uninstalling google-generativeai-0.8.5:
  Successfully uninstalled google-generativeai-0.8.5
Found existing installation: google-ai-generativelanguage 0.6.15
Uninstalling google-ai-generativelanguage-0.6.15:
  Successfully uninstalled google-ai-generativelanguage-0.6.15
INFO: pip is looking at multiple versions of google-generativeai to determine which version is compatible with other requirements. This could take a while.
ERROR: Cannot install google-ai-generativelanguage==0.7.0 and google-generativeai==0.8.5 because these package versions have conflicting dependencies.

The conflict is caused by:
    The user requested google-ai-generativelanguage==0.7.0
    google-generativeai 0.8.5 depends on google-ai-generativelanguage==0.6.15

To fix this you could try to:
1. loosen the range of package versions you've specified
2. remove package versions to allow pip to attempt to solve the dependency conflict

ERROR: ResolutionImpossible: 

In [ ]:
!pip install langgraph


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.3/153.3 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.7/216.7 kB 17.2 MB/s eta 0:00:00


In [21]:
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
from langgraph.graph import StateGraph, END
from typing import TypedDict, Optional, Dict


os.environ["GOOGLE_API_KEY"] = "YOUR_API_KEY"
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = "YOUR_API_KEY"
os.environ["LANGCHAIN_PROJECT"] = "clarify-chatbot"

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash" , temperature=0)

In [ ]:
from typing import TypedDict, Optional, Dict

class ClarifyState(TypedDict):
    query: str
    missing_info: Optional[str]
    clarification: Optional[str]
    answer: Optional[str]
    memory: Dict[str, str]

In [ ]:
def analyze_node(state: ClarifyState):
    memory_str = "\n".join([f"{k}: {v}" for k, v in state.get("memory", {}).items()])
    raw = analysis_chain.invoke({
        "query": state.get("query", ""),
        "memory_str": memory_str
    }).get("text", "")

    if raw.strip().startswith("CLARIFY:"):
        state["missing_info"] = raw.replace("CLARIFY:", "").strip()
    else:
        state["missing_info"] = None

    return state


In [ ]:
def clarify_node(state: ClarifyState):
    """Ask the user for missing info and store it into memory."""
    missing_info = state.get("missing_info")
    if missing_info:

        print("Bot:", f"Can you clarify: {missing_info}?")
        user_ans = input("You (clarification): ").strip()

        if user_ans.lower() in ["exit", "quit"]:
            print("Goodbye!")
            exit()

        state["clarification"] = user_ans

        lower_ans = user_ans.lower()
        if "my name is" in lower_ans:
            name = user_ans.split("my name is")[-1].strip().split()[0]
            state["memory"]["name"] = name
        elif "i live in" in lower_ans or "my city is" in lower_ans:
            city = user_ans.replace("i live in", "").replace("my city is", "").strip()
            state["memory"]["city"] = city
        else:

            key = missing_info.replace(" ", "_").lower()
            state["memory"][key] = user_ans

    return state


In [ ]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate


analysis_prompt = PromptTemplate(
    input_variables=["query", "memory_str"],
    template="""
Here is what you know about the user so far:
{memory_str}

Check if the user query needs clarification.
If the query can be answered with the information in memory, say "OK".
If you still need information, say:
CLARIFY: <what you need>

User query: {query}
"""
)

analysis_chain = LLMChain(llm=llm, prompt=analysis_prompt)

def analyze_node(state: ClarifyState):
    memory_str = "\n".join([f"{k}: {v}" for k, v in state["memory"].items()])

    raw_result = analysis_chain.invoke({
        "query": state["query"],
        "memory_str": memory_str
    })
    raw = raw_result["text"] if isinstance(raw_result, dict) and "text" in raw_result else str(raw_result)

    if raw.strip().startswith("CLARIFY:"):
        state["missing_info"] = raw.replace("CLARIFY:", "").strip()
    else:
        state["missing_info"] = None
    return state


answer_prompt = PromptTemplate(
    input_variables=["query", "clarification", "memory_str"],
    template="""
You are a helpful assistant. Use the user's memory and any clarifications to answer.

Memory:
{memory_str}

Clarification (if any):
{clarification}

User query:
{query}
"""
)

answer_chain = LLMChain(llm=llm, prompt=answer_prompt)

def answer_node(state: ClarifyState):
    clarification = state.get("clarification") or ""
    memory_str = "\n".join([f"{k}: {v}" for k, v in state["memory"].items()])

    ans = answer_chain.run({
        "query": state["query"],
        "clarification": clarification,
        "memory_str": memory_str
    })

    state["answer"] = ans
    print("Bot:", ans)

    text_lower = (state["query"] + " " + clarification + " " + ans).lower()
    if "my name is" in text_lower:
        name = state["query"].split("my name is")[-1].strip().split()[0]
        state["memory"]["name"] = name

    return state


In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

followup_prompt = PromptTemplate(
    input_variables=["answer"],
    template="""Suggest 3 very short follow-up questions. Number them 1, 2, 3.
Answer: {answer}"""
)

followup_chain = LLMChain(llm=llm, prompt=followup_prompt)

def suggest_followups(answer):
    suggestions_str = followup_chain.run({"answer": answer})
    suggestions = [
        line.split('.', 1)[-1].strip()
        for line in suggestions_str.split('\n')
        if line.strip()
    ]

    print("\nPossible follow-ups:")
    for i, s in enumerate(suggestions, 1):
        print(f"{i}. {s}")

    return suggestions


In [ ]:
import re

def update_memory_from_text(text, memory_dict):
    """Look for patterns like 'my name is X' or 'I live in Y' and store them."""
    lower = text.lower()

    match_name = re.search(r"my name is ([a-zA-Z ]+)", lower)
    if match_name:
        memory_dict["name"] = match_name.group(1).strip().title()

    match_city = re.search(r"(i live in|my city is) ([a-zA-Z ]+)", lower)
    if match_city:
        memory_dict["city"] = match_city.group(2).strip().title()

    return memory_dict


In [ ]:
workflow = StateGraph(ClarifyState)
workflow.add_node("analyze", analyze_node)
workflow.add_node("clarify", clarify_node)
workflow.add_node("answer", answer_node)

workflow.set_entry_point("analyze")
workflow.add_conditional_edges(
    "analyze",
    lambda s: "clarify" if s["missing_info"] else "answer",
    {"clarify": "clarify", "answer": "answer"}
)
workflow.add_edge("clarify","answer")
workflow.add_edge("answer", END)

app = workflow.compile()


In [ ]:
def clarify_with_user_memory():
    print("Type 'exit' to quit.\n")
    memory_dict = {}
    followups = []

    while True:
        if followups:
            user_query = input("You (or choose 1/2/3): ")
            if user_query.isdigit():
                idx = int(user_query)
                if 1 <= idx <= len(followups):
                    user_query = followups[idx - 1]
                    print(f"You selected follow-up: {user_query}")
        else:
            user_query = input("You: ")

        if user_query.lower() in ["exit", "quit"]:
            print("Goodbye!")
            break

        memory_dict = update_memory_from_text(user_query, memory_dict)

        state = {
            "query": user_query,
            "missing_info": None,
            "clarification": None,
            "answer": None,
            "memory": memory_dict
        }

        final_state = app.invoke(state)

        memory_dict.update(final_state["memory"])
        memory_dict = update_memory_from_text(final_state["answer"], memory_dict)

        followups = suggest_followups(final_state["answer"])

        print("Current memory:", memory_dict)


In [ ]:
clarify_with_user_memory()

Type 'exit' to quit.

You: hello
Bot: Hello! How can I help you today?

Possible follow-ups:
1. Here are 3 very short follow-up questions:
2. What's your goal?
3. What are you looking for?
4. What's the issue?
Current memory: {}
You (or choose 1/2/3): can you suggest some good coffee shop
Bot: Can you clarify: What city or area are you interested in? Do you have any preferences, like a quiet place to work, a lively atmosphere, specific coffee types, or food options??
You (clarification): delhi,nehru place
Bot: Based on your interest in Delhi, Nehru Place, here are some good coffee shop suggestions:

1.  **Starbucks:** A popular choice, known for its wide range of coffee, comfortable seating, and often a good place to work.
2.  **The Coffee Bean & Tea Leaf:** Another international chain offering a variety of coffee and tea options, often a relaxed atmosphere.
3.  **Cafe Coffee Day (CCD):** A well-known Indian chain, good for a quick coffee and snacks, usually has ample seating.
4.  **Ch